In [ ]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")
sys.path.append("../")
from lib.slices import EegSlicesChb, EegSlicesSiena, EegSlicesTusz, EegSlicesTuep
from lib.visuals import plot_eeg_windows, plot_eeg_windows_2
######### LOAD ALL THE METADATA FILES ########
slices_chb = EegSlicesChb()
slices_chb.summarize()
slices_siena = EegSlicesSiena()
slices_siena.summarize()
slices_tusz = EegSlicesTusz()
slices_tusz.summarize()
slices_tuep = EegSlicesTuep()

### * Explore the sliced EEG - CHB

In [ ]:
PATIENT = "chb01"
SEIZURE_NUMBER = 0
PERIOD = [110, 130]

metadata, eeg_array = slices_chb.get(PATIENT, SEIZURE_NUMBER)
current_metadata = {}
current_metadata["seizures"] = [(0, 0, None), (metadata["seizure_start"], metadata["seizure_end"], metadata["seizure_type"])]
plot_eeg_windows(eeg_array, current_metadata, metadata["channels"], metadata["sampling_frequency"], PERIOD)

### * Generate EEG images - CHB

In [ ]:
PERIOD = [50, 150]

base_path = os.getenv("BIOMARKERS_PROJECT_HOME")
for patient, seizures in slices_chb._metadata.items():
    for idx, _ in enumerate(seizures):
        current_metadata = {}
        metadata, eeg_array = slices_chb.get(patient, idx)
        current_metadata["seizures"] = [(0, 0, None), (metadata["seizure_start"], metadata["seizure_end"], metadata["seizure_type"])]
        output = output_file = os.path.join(base_path, "images", "eegs", "chb", f"{patient}_{idx}.png")
        plot_eeg_windows(eeg_array, current_metadata, metadata["channels"], metadata["sampling_frequency"], PERIOD, output)
        break

### * Explore the sliced EEG - SIENA

In [ ]:
PATIENT = "PN14"
SEIZURE_NUMBER = 0
PERIOD = [110, 140]

metadata, eeg_array = slices_siena.get(PATIENT, SEIZURE_NUMBER)
current_metadata = {}
current_metadata["seizures"] = [(0, 0, None), (metadata["seizure_start"], metadata["seizure_end"], metadata["seizure_type"])]
plot_eeg_windows(eeg_array, current_metadata, metadata["channels"], metadata["sampling_frequency"], PERIOD)

### * Generate EEG images - SIENA

In [ ]:
PERIOD = [50, 150]

base_path = os.getenv("BIOMARKERS_PROJECT_HOME")
for patient, seizures in slices_siena._metadata.items():
    for idx, _ in enumerate(seizures):
        current_metadata = {}
        metadata, eeg_array = slices_siena.get(patient, idx)
        current_metadata["seizures"] = [(0, 0, None), (metadata["seizure_start"], metadata["seizure_end"], metadata["seizure_type"])]
        output = output_file = os.path.join(base_path, "images", "eegs", "siena", f"{patient}_{idx}_{metadata['seizure_type']}.png")
        plot_eeg_windows(eeg_array, current_metadata, metadata["channels"], metadata["sampling_frequency"], PERIOD, output)
        break

### * Explore the sliced EEG - TUSZ

In [ ]:
PATIENT = "aaaaatjz"
SEIZURE_NUMBER = 0
PERIOD = [0, 80]

metadata, eeg_array = slices_tusz.get(PATIENT, SEIZURE_NUMBER)

###########################################################
current_metadata = {}
current_metadata["seizures"] = [(0, 0, None), (metadata["seizure_start"], metadata["seizure_end"], metadata["seizure_type"])]
plot_eeg_windows(eeg_array, current_metadata, metadata["channels"], metadata["sampling_frequency"], PERIOD)

### * Generate EEG images - TUSZ

In [ ]:
PERIOD = [50, 150]

base_path = os.getenv("BIOMARKERS_PROJECT_HOME")
for patient, seizures in slices_tusz._metadata.items():
    for idx, _ in enumerate(seizures):
        current_metadata = {}
        metadata, eeg_array = slices_tusz.get(patient, idx)
        current_metadata["seizures"] = [(0, 0, None), (metadata["seizure_start"], metadata["seizure_end"], metadata["seizure_type"])]
        output = output_file = os.path.join(base_path, "images", "eegs", "tusz", f"{patient}_{idx}_{metadata['seizure_type']}.png")
        plot_eeg_windows(eeg_array, current_metadata, metadata["channels"], metadata["sampling_frequency"], PERIOD, output)
        break

### * Explore the sliced EEG - TUEP

In [ ]:
PATIENT = "aaaaapwy"
SLICE_NUMBER = 1
PERIOD = [50, 55]

metadata, eeg_array = slices_tuep.get(PATIENT, SLICE_NUMBER)

###########################################################
current_metadata = {}
current_metadata["seizures"] = [(0, 0, None)]
plot_eeg_windows(eeg_array, current_metadata, metadata["channels"], metadata["sampling_frequency"], PERIOD)